In [ ]:
# Code taken from https://colab.research.google.com/drive/1ow8ePMoYAhBKyKOuH1JHmxPYFvOteBQH?usp=sharing#scrollTo=cQRhLUHFAmU6
# Code taken from https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-from-scratch-for-mnist-handwritten-digit-classification/

# Import libraries
from matplotlib import pyplot as plt
from matplotlib.legend_handler import HandlerLineCollection, HandlerTuple
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import Accuracy, Precision, Recall, F1Score
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from numpy import mean, std
import pandas as pd

In [ ]:
def load_data():
    data = pd.read_csv('digit-recognizer/train.csv')
    x = data.loc[:, data.columns != "label"]
    y = data['label']
    # Split train and test data
    xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state = 42)
    # Reshape data
    xtrain = xtrain.values.reshape((xtrain.shape[0], 28, 28, 1))
    xtest = xtest.values.reshape((xtest.shape[0], 28, 28, 1))
    ytrain = np.array(ytrain)
    ytest = np.array(ytest)
    # Normalize x
    xtrain_norm = xtrain.astype('float32') / 255.0
    xtest_norm = xtest.astype('float32') / 255.0
    # Make labels categorical
    ytrain = to_categorical(ytrain)
    ytest = to_categorical(ytest)
    return xtrain_norm, ytrain, xtest_norm, ytest

In [ ]:
# Define CNN model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), padding = "same", activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(BatchNormalization())
	model.add(Dense(10, activation='softmax'))
	opt = SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy', Precision, Recall, F1Score])
	return model

In [ ]:
# Evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	# Prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# Enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		# Define model
		model = define_model()
		# Select rows for train and test
		trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		# Fit model
		history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
		# Evaluate model
		_, acc, prec, rec, f1 = model.evaluate(testX, testY, verbose=0)
		prec = mean(prec)
		print('Accuracy > %.3f' % (acc * 100.0))
		# print('Precision > %.3f' % (prec * 100.0))
		# print('Recall > %.3f' % (rec * 100.0))
		# print('F1 Score > %.3f' % (f1 * 100.0))
		# Store scores
		scores.append([acc, prec, rec, f1])
		histories.append(history)
	return scores, histories, model

In [ ]:
# Plot diagnostic learning curves
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# Plot loss
		plt.title('Cross Entropy Loss')
		plt.plot(histories[i].history['loss'], color='blue', label='train')
		plt.plot(histories[i].history['val_loss'], color='orange', label='test')
		plt.xticks([0, 2, 4, 6, 8], [1, 2, 3, 4, 5])
		plt.xlabel("Fold K")
		plt.ylabel("Loss")
	plt.legend(loc="upper right", handler_map={tuple: HandlerTuple(ndivide=None)})
	plt.show()
	for i in range(len(histories)):
		# Plot accuracy
		plt.title('Classification Accuracy')
		plt.plot(histories[i].history['accuracy'], color='blue', label='train')
		plt.xticks([0, 2, 4, 6, 8], [1, 2, 3, 4, 5])
		plt.plot(histories[i].history['val_accuracy'], color='orange', label='test')
		plt.xlabel("Fold K")
		plt.ylabel("Accuracy")
	plt.legend(loc="lower right", handler_map={tuple: HandlerTuple(ndivide=None)})
	plt.show()

In [ ]:
# Summarize model performance
def summarize_performance(scores):
	print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores[:, 0])*100, std(scores[:, 0])*100, len(scores[:, 0])))
	print('Precision: mean=%.3f std=%.3f, n=%d' % (mean(scores[:, 1])*100, std(scores[:, 1])*100, len(scores[:, 1])))
	print('Recall: mean=%.3f std=%.3f, n=%d' % (mean(scores[:, 2])*100, std(scores[:, 2])*100, len(scores[:, 2])))
	print('F1 Score: mean=%.3f std=%.3f, n=%d' % (mean(scores[:, 3])*100, std(scores[:, 3])*100, len(scores[:, 3])))
	plt.boxplot(scores)
	plt.xticks([1, 2, 3, 4], ['Accuracy', 'Precision', 'Recall', 'F1 Score'])
	plt.xlabel("Metric")
	plt.ylabel("Performance")
	plt.show()

In [ ]:
class estimator:
    _estimator_type = ''
    classes_=[]
    def __init__(self, model, classes):
        self.model = model
        self._estimator_type = 'classifier'
        self.classes_ = classes
    def predict(self, X):
        y_prob= self.model.predict(X)
        y_pred = y_prob.argmax(axis=1)
        return y_pred

In [ ]:
# Run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_data()
	# evaluate model
	scores, histories, model = evaluate_model(trainX, trainY)
	# learning curves
	summarize_diagnostics(histories)
	# summarize estimated performance
	summarize_performance(np.array(scores))
	# Get results from test set
	classifier = estimator(model, list(range(0, 10)))
	predictedY = classifier.predict(testX)
	testY = np.array(pd.from_dummies(pd.DataFrame(testY)))
	print(f"Accuracy: {round(accuracy_score(testY, predictedY), 4) * 100}%")
	print(f"Precision: {round(precision_score(testY, predictedY, average = 'macro'), 4) * 100}%")
	print(f"Recall: {round(recall_score(testY, predictedY, average = 'macro'), 4) * 100}%")
	print(f"F1 Score: {round(f1_score(testY, predictedY, average = 'macro'), 4) * 100}%")
	ConfusionMatrixDisplay.from_estimator(estimator=classifier, X = testX, y = testY)

In [ ]:
run_test_harness()

In [ ]:
model = define_model()
plot_model(model, "cnn_model.png")